In [1]:
import numpy as np
import pandas as pd
import nltk
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [2]:
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

In [3]:
from string import punctuation
stopwords_en.append(set(punctuation))
print(stopwords_en)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
def extract_entity_name(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(" ".join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_name(child))
    
    return entity_names

In [36]:
ip_feat = []
op_cols = [-1, 0, 1]

In [37]:
for file_name in ['8-K 2015 Jan.html', '8-K 2015.html', '8-K 2016.html', 'msft_8k.html']:
    with open(file_name, 'r', encoding="utf-8") as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'html')

    # tokens = nltk.word_tokenize(soup.get_text())

    sent_token = nltk.sent_tokenize(soup.get_text())

    tagged_sentences = []
    for i in sent_token:
        wordslist = nltk.word_tokenize(i)
        wordslist = [w for w in wordslist if not w in stopwords_en]
        tagged = nltk.pos_tag(wordslist)
        tagged_sentences.append(tagged)
        # print(tagged)


    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    print(chunked_sentences)

    entity_names = []
    for tree in chunked_sentences:
        entity_names.extend(extract_entity_name(tree))
    # print(entity_names)

    s = " "
    ip_data = s.join(entity_names)
    ip_feat.append(ip_data)

<generator object ParserI.parse_sents.<locals>.<genexpr> at 0x000002DF493F0270>
<generator object ParserI.parse_sents.<locals>.<genexpr> at 0x000002DF50B04820>
<generator object ParserI.parse_sents.<locals>.<genexpr> at 0x000002DF4971E9E0>
<generator object ParserI.parse_sents.<locals>.<genexpr> at 0x000002DF50B04820>


In [38]:
print(len(ip_feat))

4


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier

In [39]:
tfidf_obj = TfidfVectorizer()
dummy_obj = tfidf_obj.fit(ip_feat)
tfidf_feature_train = tfidf_obj.transform(ip_feat[:-1])
tfidf_feature_test = tfidf_obj.transform(ip_feat[-1:])

In [41]:
print(tfidf_feature_train.shape)

(3, 225)


In [43]:
mlp = MLPClassifier(hidden_layer_sizes=(8, 8, 8), activation='relu', solver='adam', max_iter=500)
mlp.fit(tfidf_feature_train,op_cols)

C:\Users\nirmi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(8, 8, 8), max_iter=500)

In [46]:
print(tfidf_feature_test.shape)

(1, 225)


In [51]:
# tfidf_feature_test = tfidf_obj.predict(tfidf_feature_test)
print(mlp.predict(tfidf_feature_test))

[-1]
